In [32]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import re,os
import glob, traceback
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [33]:
# !nvidia-smi

## 0. Read all csv files

In [2]:
dff = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2020_emo.csv')
dff.shape

(3502171, 19)

In [34]:
dff.drop_duplicates(subset=['text', 'date'], inplace=True)
dff.shape

(2147729, 19)

In [4]:
# dff= dff[:10000]

In [5]:
dff = dff.loc[:, ~dff.columns.str.match('Unnamed')]

In [6]:
dff.sample(2)

,id,created_at,text,user_id,name,screen_name,description,DrugAbuse,lang,gender,age,org,sentiment,sent_score,date,is_teenage,drug_type,emotion
1263938,1256422838900326401,Sat May 02 03:18:42 +0000 2020,USER border interceptor border patrol seize ne...,2165719802,CHRIS PEAK,cpskins44,BIG WASHINGTON REDSKINS and LOUISVILLE CARDINA...,1,en,male,>=40,non-org,pos,0.4404,2020-05-01,0,['stimulants'],"['anger', 'disgust']"
107738,1221395220530679808,Sun Jan 26 11:31:27 +0000 2020,USER legs high HTTPURL,1205336912111833088,chico87,chico0287,Pasarla bien,1,en,male,30-39,is-org,neu,0.0000,2020-01-01,0,[],[]


In [7]:
dff.shape

(2147729, 18)

In [11]:
df = dff[:100]

In [34]:
import scipy.stats as stats

# Prepare data for one-way ANOVA
anova_data = [df[df['org'] == group]['sent_score'] for group in df['org'].unique()]

# Perform one-way ANOVA
f_statistic, p_value = stats.f_oneway(*anova_data)

print(f"One-way ANOVA - Gender: F-statistic: {f_statistic}, P-value: {p_value}")

One-way ANOVA - Gender: F-statistic: 2.44462978868458, P-value: 0.12115127441946298


In [35]:
anova_data

[0      0.0000
 1      0.0000
 2     -0.3400
 3      0.2263
 5      0.0000
         ...  
 122    0.0000
 124    0.3182
 125    0.0000
 126   -0.6486
 127    0.0000
 Name: sent_score, Length: 93, dtype: float64,
 4      0.0000
 14     0.2732
 26     0.0000
 62     0.7003
 67     0.0000
 96     0.0000
 114    0.0000
 Name: sent_score, dtype: float64]

In [32]:
f_statistic, p_value = stats.f_oneway(*anova_data)
print(f"One-way ANOVA - Age Group: F-statistic: {f_statistic}, P-value: {p_value}")

One-way ANOVA - Age Group: F-statistic: nan, P-value: nan


In [57]:
# Create samples for each theme
# sample_size = 100  # Define a sample size
samples = {age: dff[dff['age'] == age] for age in  dff['age'].unique()}

# Extract counts
sample_counts = {age: sample.groupby('DrugAbuse').size() for age, sample in samples.items()}

# Convert to a list of lists for ANOVA
anova_data = [list(counts) for counts in sample_counts.values()]

print(*anova_data)


[457546] [734455] [467816] [487912]


In [58]:
import numpy as np
import scipy.stats as stats

# Prepare data for ANOVA
values = anova_data

# Perform ANOVA
f_statistic, p_value = stats.f_oneway(*values)

print(f"F-statistic: {f_statistic}")
print(f"P-value: {p_value}")


F-statistic: nan
P-value: nan


/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4102: DegenerateDataWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  if _f_oneway_is_too_small(samples):


## 1. Stats 

In [35]:
is_org = dff[dff['org'] =='is-org']
non_org = dff[dff['org'] !='is-org']

In [36]:
total = dff.shape[0]
total

2147729

In [37]:
total_tweets = dff.shape[0]
count_is_org = is_org.shape[0]
count_non_org = non_org.shape[0]

# Calculating percentages
percent_is_org = (count_is_org / total_tweets) * 100
percent_non_org = (count_non_org / total_tweets) * 100

# Printing the results
print(f"Total Drug Abuse Identified tweets in 2020 = {total_tweets}")
print(f"Org = {count_is_org} ({percent_is_org:.2f}%)")
print(f"Non Org = {count_non_org} ({percent_non_org:.2f}%)")

Total Drug Abuse Identified tweets in 2020 = 2147729
Org = 221815 (10.33%)
Non Org = 1925914 (89.67%)


In [38]:
total_user_tweets = total_tweets # non_org.shape[0]
total_female_tweets = non_org[(non_org['gender'] == 'female')].shape[0]
total_male_tweets = non_org[(non_org['gender'] == 'male')].shape[0]
percent_female = (total_female_tweets / total_user_tweets) * 100
percent_male = (total_male_tweets / total_user_tweets) * 100

# Printing the results
print(f"Total Drug Abuse Identified tweets by non-org users in 2020 = {total_user_tweets}")
print(f"Female = {total_female_tweets} ({percent_female:.2f}%)")
print(f"Male = {total_male_tweets} ({percent_male:.2f}%)")

Total Drug Abuse Identified tweets by non-org users in 2020 = 2147729
Female = 865271 (40.29%)
Male = 1060643 (49.38%)


In [39]:
total_users_tweets = total_tweets #non_org.shape[0]
total_teenagers_tweets = non_org[(non_org['is_teenage'] == 1)].shape[0]
total_adult_tweets = non_org[(non_org['is_teenage'] != 1)].shape[0]
percent_teenage = (total_teenagers_tweets / total_user_tweets) * 100
percent_adult = (total_adult_tweets / total_user_tweets) * 100

# Printing the results
print(f"Total Drug Abuse Identified tweets by non-org users in 2020 = {total_user_tweets}")
print(f"Teenage = {total_teenagers_tweets} ({percent_teenage:.2f}%)")
print(f"Adult = {total_adult_tweets} ({percent_adult:.2f}%)")

Total Drug Abuse Identified tweets by non-org users in 2020 = 2147729
Teenage = 706511 (32.90%)
Adult = 1219403 (56.78%)


In [40]:
tf= non_org[(non_org['is_teenage'] == 1) & (non_org['gender'] == 'female')].shape[0]
tm= non_org[(non_org['is_teenage'] == 1) & (non_org['gender'] == 'male')].shape[0]
af= non_org[(non_org['is_teenage'] != 1) & (non_org['gender'] == 'female')].shape[0]
am= non_org[(non_org['is_teenage'] != 1) & (non_org['gender'] == 'male')].shape[0]
percent_tf = (tf / total_user_tweets) * 100
percent_tm = (tm / total_user_tweets) * 100
percent_af = (af / total_user_tweets) * 100
percent_am = (am / total_user_tweets) * 100

print(f"Teenage and Female = {tf} ({percent_tf:.2f}%)")
print(f"Teenage and Male = {tm} ({percent_tm:.2f}%)")
print(f"Adult and Female = {af} ({percent_af:.2f}%)")
print(f"Adult and Male = {am} ({percent_am:.2f}%)")

Teenage and Female = 377487 (17.58%)
Teenage and Male = 329024 (15.32%)
Adult and Female = 487784 (22.71%)
Adult and Male = 731619 (34.06%)


In [41]:
dff.sample(1)

,id,created_at,text,user_id,name,screen_name,description,DrugAbuse,lang,gender,age,org,sentiment,sent_score,date,is_teenage,drug_type,emotion,cleaned_text
1237250,1257045915371782145,Sun May 03 20:34:35 +0000 2020,woke bestfriend beer bong white claw say bestf...,498418839,o l i v i a 🥺,icedteajc,one of a kind 🦋,1,en,female,<=18,non-org,pos,0.2732,2020-05-01,1,['alcohol'],"['joy', 'optimism']",woke bestfriend beer bong white claw say bestf...


## 1.1 By Sentiment

In [42]:
total_users_tweets = total_tweets #non_org.shape[0]
total_neu_tweets = non_org[(non_org['sentiment'] == 'neu')].shape[0]
total_pos_tweets = non_org[(non_org['sentiment'] == 'pos')].shape[0]
total_neg_tweets = non_org[(non_org['sentiment'] == 'neg')].shape[0]
percent_neu = (total_neu_tweets / total_user_tweets) * 100
percent_pos = (total_pos_tweets / total_user_tweets) * 100
percent_neg = (total_neg_tweets / total_user_tweets) * 100

# Printing the results
print(f"Total Drug Abuse Identified tweets by non-org users in 2020 = {total_user_tweets}")
print(f"Neu = {total_neu_tweets} ({percent_neu:.2f}%)")
print(f"Pos = {total_pos_tweets} ({percent_pos:.2f}%)")
print(f"Neg = {total_neg_tweets} ({percent_neg:.2f}%)")

Total Drug Abuse Identified tweets by non-org users in 2020 = 2147729
Neu = 691110 (32.18%)
Pos = 578192 (26.92%)
Neg = 656612 (30.57%)


In [53]:
dff.shape

(2147729, 19)

In [54]:
org.shape

NameError: name 'org' is not defined

In [52]:
non_org.shape

(1925914, 19)

In [43]:
teenage = non_org[(non_org['is_teenage'] == 1)]
adult = non_org[(non_org['is_teenage'] != 1)]

In [51]:
teenage = teenage[teenage['drug_type'] != '[]']
teenage.shape

(224945, 19)

## 2. Analysis User demographics

In [14]:
non_org['user_id'].nunique()

915

In [15]:
udf = non_org.drop_duplicates(subset=['user_id'], keep='first')
udf.shape

(915, 17)

In [17]:
total_users = udf.shape[0]
tf= udf[(udf['is_teenage'] == 1) & (udf['gender'] == 'female')].shape[0]
tm= udf[(udf['is_teenage'] == 1) & (udf['gender'] == 'male')].shape[0]
af= udf[(udf['is_teenage'] != 1) & (udf['gender'] == 'female')].shape[0]
am= udf[(udf['is_teenage'] != 1) & (udf['gender'] == 'male')].shape[0]
percent_tf = (tf / total_users) * 100
percent_tm = (tm / total_users) * 100
percent_af = (af / total_users) * 100
percent_am = (am / total_users) * 100

print(f"Cohort I: Teenage and Female = {tf} ({percent_tf:.2f}%)")
print(f"Cohort II: Teenage and Male = {tm} ({percent_tm:.2f}%)")
print(f"Cohort III: Adult and Female = {af} ({percent_af:.2f}%)")
print(f"Cohort IV: Adult and Male = {am} ({percent_am:.2f}%)")

Cohort I: Teenage and Female = 241 (26.34%)
Cohort II: Teenage and Male = 137 (14.97%)
Cohort III: Adult and Female = 256 (27.98%)
Cohort IV: Adult and Male = 281 (30.71%)


## Now Analysis of Tweets by Drug type in each cohort.

In [18]:
tf_df = non_org[(non_org['is_teenage'] == 1) & (non_org['gender'] == 'female')]
tm_df = non_org[(non_org['is_teenage'] == 1) & (non_org['gender'] == 'male')]
af_df = non_org[(non_org['is_teenage'] != 1) & (non_org['gender'] == 'female')]
am_df = non_org[(non_org['is_teenage'] != 1) & (non_org['gender'] == 'male')]
tf_df.shape, tm_df.shape, af_df.shape, am_df.shape

((241, 17), (137, 17), (256, 17), (281, 17))

In [19]:
# print(f'Teenager:: \n Total tweets by Female: { tf_df.shape[0]} \n', tf_df['drug_type'].value_counts(), f'Total tweets by Male: { tm_df.shape[0]} \n', tm_df['drug_type'].value_counts())
# print(f'\n Adult:: \n Total tweets by Female: { af_df.shape[0]} \n', af_df['drug_type'].value_counts(), f'Total tweets by Male: { am_df.shape[0]} \n', am_df['drug_type'].value_counts())

## LDA

In [21]:
# tf_df.shape
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=0.75, min_df=0.001, use_idf=True)
posts_tfidf_bowdff = vectorizer.fit_transform(dff['cleaned_text'].values.astype('U'))

In [22]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top = lda_model.fit_transform(posts_tfidf_bowdff)

In [ ]:
# vocab = vectorizer.get_feature_names()
vocab = vectorizer.get_feature_names_out()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
     print("Topic "+str(i)+": ")
     # print(sorted_words)
     for t in sorted_words:
        print(t[0], end=" ")
     print('\n')

## BERT Topic

In [8]:
from bertopic import BERTopic

/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/umap/d

In [9]:
words_to_remove = ['httpurl', 'hashtag', 'user']
dff['cleaned_text'] = dff['text'].apply(lambda x: ' '.join(['' if word.lower() in words_to_remove else word for word in x.split()]))

In [19]:
# sample = dff[dff['sentiment']=='neg']
sample = dff.sample(10000)

In [20]:
sample.shape

(10000, 19)

In [21]:
# sample = sample[:10000]

In [22]:
docs = sample.cleaned_text.tolist()

In [23]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:07<00:00, 42.80it/s]


In [24]:
from umap import UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [25]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
# stop_words = stop_words + ['httpurl', 'hashtag', 'user']
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

In [27]:
import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
# pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
openai.api_key = "sk-proj-V9eUaxTcHBwbuk09RvpHT3BlbkFJiONTDMtF1Nb9SWIfmWIn"
prompt = """
I have a topic that contains the following documents: 
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
openai_model = OpenAI(model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    # "POS": pos_model
}


In [29]:
topic_model = BERTopic(
 # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)


In [30]:
# Train model
topics, probs = topic_model.fit_transform(docs, embeddings)

2024-06-14 12:23:59,119 - BERTopic - Reduced dimensionality
2024-06-14 12:23:59,846 - BERTopic - Clustered reduced embeddings


AttributeError: module 'openai' has no attribute 'error'

In [ ]:
# print(topic_model.topics_)

In [ ]:
freq = topic_model.get_topic_info()
freq.shape

In [ ]:
freq

In [ ]:
# topic_model.get_topic(-1)

In [ ]:
topic_model.visualize_barchart(top_n_topics=12)

In [ ]:
dff.shape

In [ ]:
freq.iloc[-1]['Representative_Docs']

In [ ]:
new_data = pd.Series(topics, name=int)
df = dff[['id','date', 'text', 'user_id', 'age', 'is_teenage','gender','org','sentiment','emotion', 'cleaned_text', 'topic' ]]
df['topic'] = new_data

In [ ]:
df[df['topic']==-1]

In [32]:
topic_model.reduce_topics(docs, nr_topics=2)